## 5. 데이터 정제 및 스케일링 자동화
타겟변수와의 correlation이 더 높은 파생변수의 생성, 데이터 정제 및 스케일링을 자동화 및 축약 시켜놓으면 코드의 길이가 짧아지고 가독성이 좋아진다. pipeline 및 columntransformer의 사용에 대해 알아보자.

### (1) 파생변수 자동생성 전처리기 구현
sklearn의 fit, transform, predict 구현철학을 그대로 가져와 전처리 역할을 자동화 해주는 클래스를 사전에 구현해놓으면 간단한 객체생성만으로도 데이터셋에 파생변수를 빠르게 집어넣을 수 있다. sklearn.base의 BaseEstimator을 상속하면 get_params,set_params 메서드를 사용할 수 있으며 TransformerMixin을 상속하면 fit_transform메서드를 사용할 수 있다. 

 *get_params : 세팅할 수 있는 하이퍼파라미터 목록을 출력하는 메서드
 
 *set_params : 하이퍼파라미터를 직접 변경하는 메서드

> ℹ️ **부가 설명 | sklearn 구현철학**
 - fit() : 데이터에 필요한 통계수치등을 학습하여 내부 객체에 저장
 - transform() : fit()으로 얻은 수치들을 실제 데이터셋에 적용
 - predict() : 학습된 모델 파라미터를 이용해서 입력값 X에 대한 예측값 y를 출력

In [82]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y = None):
        return self
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room :
            bedrooms_per_household = X[:, bedrooms_ix] / X[:, households_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_household]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

In [83]:
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room = False)
housing_extra_attr = attr_adder.transform(housing.values)

### (2) 데이터 스케일링 및 파이프라인을 통한 자동화
수치형 데이터들의 범위가 제각각이면 모델 학습시 어려움이 있을 수 있다. 이를 방지하기 위해 sklearn.preprocessing의 min-max-scaling, standardization을 사용할 수 있다. 아래 예시에서는 StandardScaler을 통해 모든 데이터들의 값 범위를 0~1로 제한한다. 또한 sklearn.pipeline의 Pipeline을 통해 Imputer객체를 통한 결측치 대체, 파생변수 생성, 스케일링을 자동화할 수 있다. Pipeline객체의 fit_transform메서드를 실행하면 파라미터 데이터셋에 데이터정제 및 스케일링이 자동적용된다.

In [84]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy = 'median')),
    ('attr_adder', CombinedAttributesAdder(add_bedrooms_per_room = False)),
    ('std_scaler', StandardScaler()),
])
housing_num_tr = num_pipeline.fit_transform(housing_num)

### (3) 문자 범주형 데이터 전처리 자동화
sklearn.compose의 ColumnTransformer객체를 통해 문자 범주형 데이터에 대한 전처리 또한 자동화할 수 있다. 데이터셋의 column명들을 수치형, 문자형으로 나누어 리스트화 시킨 후 ColumnTransformer객체를 통해 수치형 데이터의 전처리 뿐만 아니라 문자형 데이터의 전처리 또한 간단하게 처리할 수 있다.

In [85]:
from sklearn.compose import ColumnTransformer

num_attrib = list(housing_num)
cat_attrib = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attrib), 
    ('cat', OneHotEncoder(), cat_attrib)
])

housing_prepared = full_pipeline.fit_transform(housing)